In [ ]:
# dependencies import
from common_dependencies import *
from typing import NamedTuple
import logging
logger = logging.getLogger('main.model_test.ipynb')
# детерминация случайных величин, отвечающих за выбор первоначальных весов и биасов
tf.compat.v1.set_random_seed(290)
tf.random.set_seed(290)

In [ ]:
# paths for files with original data
PATH_TO_DATA = {
    'run_1': 
        ('data/original_data/run_1/run1_WM32_data.csv',
        'data/original_data/run_1/run1_WM32_defects.csv',
        'data/original_data/run_1/run1_WM32_pipe.csv'),
    'run_2':
        ('data/original_data/run_2/run2_WM32_data.csv',
        'data/original_data/run_2/run2_WM32_defects.csv',
        'data/original_data/run_2/run2_WM32_pipe.csv')
}

PATH_TO_MODEL = 'networks/CNN'

RUNS = [1,2]
MODEL_VER = '10'
MODEL_NUM = '01'
XSHIFT = 200

for name in os.listdir(PATH_TO_MODEL):
    res = re.match(F'(id=v{MODEL_VER}n{MODEL_NUM}).*', name)
    if not res is None:
        PATH_TO_MODEL += '/' + res[0]
        break

DataCrop = NamedTuple("DataCrop", [('top', int), ('bottom', int),('left', int), ('right', int)])

In [ ]:
def get_dataset_gen(path_to_data_tuple: tuple(),
                x_shift: int,
                data_part: DataCrop,
                crop_size: int, 
                crop_step: int):
    logger.debug("start")
    x_df, y_df = dw.get_x_and_y_data(*path_to_data_tuple)
    #dw.draw_defects_map(y_df, title=f'Read defects map')

    x_df = dw.roll_df(x_df, x_shift, 1)
    y_df = dw.roll_df(y_df, x_shift, 1)
    #dw.draw_defects_map(y_df, title=f'The shifted by {x_shift} on x axis read defects map')
    
    x_df = x_df.iloc[data_part.top:data_part.bottom, data_part.left:data_part.right]
    y_df = y_df.iloc[data_part.top:data_part.bottom, data_part.left:data_part.right]
    #dw.draw_defects_map(y_df, title=f'The part took for learning')
    
    x_df = dw.extend_df_for_crops_dividing(x_df, crop_size, crop_step)
    y_df = dw.extend_df_for_crops_dividing(y_df, crop_size, crop_step)
    #dw.draw_defects_map(y_df, title=f'The part after extending for crop dividing')

    x_arr = dw.df_to_numpy(x_df)
    y_arr = y_df.to_numpy()
    
    x_arr_time = dw.standardize_data(x_arr[:,:,:32])
    x_arr_amp = dw.standardize_data(x_arr[:,:,32:])
    y_arr = dw.standardize_data(y_arr)
    
    x_arr_time_crops_gen = dw.get_augmented_crop_generator(x_arr_time, crop_size, crop_step)
    x_arr_amp_crops_gen = dw.get_augmented_crop_generator(x_arr_amp, crop_size, crop_step)
    
    y_data_binary_gen = (1 if np.sum(crop > 0) else 0 for crop in 
                         dw.get_augmented_crop_generator(y_arr, crop_size, crop_step))
    y_data_depth_gen = (np.max(crop) for crop in 
                        dw.get_augmented_crop_generator(y_arr, crop_size, crop_step))

    logger.debug("end")
    return x_arr_time_crops_gen, x_arr_amp_crops_gen, y_data_binary_gen, y_data_depth_gen

In [ ]:
# Загрузка модели
load_model = keras.models.load_model(PATH_TO_MODEL)
CROP_SIZE = load_model.inputs[0].shape[1]
CROP_STEPS = [CROP_SIZE, CROP_SIZE//2, CROP_SIZE//4]

In [ ]:
# тест модели
res_list = list()
for crop_step in CROP_STEPS:
    (test_x_time_gen, 
     test_x_amp_gen, 
     test_y_binary_gen,
     test_y_depth_gen) = get_dataset_gen(PATH_TO_DATA[f'run_{run}'], XSHIFT, DataCrop(None,None,None,200), CROP_SIZE, 20)
    
    test_x_time = np.stack([crop for crop in test_x_time_gen])
    test_x_amp = np.stack([crop for crop in test_x_amp_gen])
    test_y_binary = np.array([binary for binary in test_y_binary_gen])
    test_y_depth = np.array([depth for depth in test_y_depth_gen])
    
    res = load_model.evaluate([test_x_time, test_x_amp], test_y_binary, batch_size = 32, return_dict=True)
    res_list.append({'res':res,'crop_size':CROP_SIZE,'crop_step':crop_step})

In [ ]:
for line in res_list:
    print(f'crop_size: {line["crop_size"]}')
    print(f'crop_size: {line["crop_step"]}')
    print(line['res'])
    print()